# Deep Learning - Exercise 2

This lecture is about introduction to using ANN for regression tasks.

We will use our models on [Auto MPG](https://archive.ics.uci.edu/ml/datasets/auto+mpg) dataset.

This dataset contains fule consumptions of several vehicles in miles per gallon. So, we need to predict the fuel efficiencies of various vehicles from the data that has been provided.

[Open in Google colab](https://colab.research.google.com/github/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/dl_02.ipynb)
[Download from Github](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/dl_02.ipynb)

##### Remember to set **GPU** runtime in Colab!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # plotting
import seaborn as sns # plotting
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, PowerTransformer

tf.version.VERSION

In [ ]:
"""
Computes MAPE
"""
def mean_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

"""
Computes SMAPE
"""
def symetric_mean_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.mean(np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0))) * 100

"""
Computes MAE, MSE, MAPE, SMAPE, R2
"""
def compute_metrics(df: pd.DataFrame) -> pd.DataFrame:
    y_true, y_pred = df['y_true'].values, df['y_pred'].values
    return compute_metrics_raw(y_true, y_pred)

def compute_metrics_raw(y_true: pd.Series, y_pred: pd.Series) -> pd.DataFrame:
    mae, mse, mape, smape, r2 = mean_absolute_error(y_true=y_true, y_pred=y_pred), mean_squared_error(y_true=y_true, y_pred=y_pred), mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred), symetric_mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred), r2_score(y_true=y_true, y_pred=y_pred)
    return pd.DataFrame.from_records([{'MAE': mae, 'MSE': mse, 'MAPE': mape, 'SMAPE': smape, 'R2': r2}], index=[0])

In [ ]:
def show_history(history):
    plt.figure()
    for key in history.history.keys():
        plt.plot(history.epoch, history.history[key], label=key)
    plt.legend()
    plt.tight_layout()

In [ ]:
def show_history_loss(history):
    plt.figure()
    for key in history.history.keys():
        if 'loss' not in key:
            continue
        plt.plot(history.epoch, history.history[key], label=key)
    plt.legend()
    plt.tight_layout()

## 🔎 Let's find answers to these quetions first! 🔎
1) How is regression different from the classification task?
2) How would you solve regression tasks? 
    - Describe briefly the pipeline, model and metrics used
2) How is ANN different from Linear regression model?

# Load the dataset first

## Dataset info
* Number of Instances: 398
* Number of Attributes: 9 including the class attribute

**Attribute Information:**

    1. mpg:           continuous
        * miles per gallon - higher the better
    2. cylinders:     multi-valued discrete
    3. displacement:  continuous
    4. horsepower:    continuous
    5. weight:        continuous
    6. acceleration:  continuous
    7. model year:    multi-valued discrete
    8. origin:        multi-valued discrete
    9. car name:      string (unique for each instance)

* Missing Attribute Values:  horsepower has 6 missing values

In [ ]:
url = 'https://raw.githubusercontent.com/rasvob/VSB-FEI-Deep-Learning-Exercises/main/datasets/auto-mpg.csv'
rel_path = 'datasets/auto-mpg.csv'
df = pd.read_csv(url, na_values='?', sep=';')

In [ ]:
df

## Check missing values

In [ ]:
df.isna().sum()

## 📊 Plot the data first 
* Which row/columns is the most important for us? Why? 📈
* Can you identify **categorical** features from these plot? How?
* Are the numeric ranges of the features the same? 
    * Can the difference among it make our task harder?
* What about other features? Is there any covariance/colinearity?

In [ ]:
sns.pairplot(df)

## Do you see any colinearity in the data?
* Can it cause any issue? How to deal with it?

In [ ]:
sns.heatmap(df.corr(), cmap='Greens', annot=True)

## We can plot the categorical data using boxplots
* Beware that the data are about cars from 80s, we won't see many 6 or 8 cylinder cars nowadays

In [ ]:
sns.boxplot(data = df, x='cylinders', y='mpg')

In [ ]:
sns.boxplot(data = df, x='origin', y='mpg')

## 💡 There is no info about the *origin* feature = detective work incoming 🙂

### What do you think that the origin means based on the printed data?
* And what car origin is your favourite? 🙂

In [ ]:
df.loc[df.origin == 1, 'car_name']

In [ ]:
df.loc[df.origin == 2, 'car_name']

In [ ]:
df.loc[df.origin == 3, 'car_name']

## Okay, now we have the basic understanding of the data we can start to try some models
* We need to deal with the NA values first, as is it just a few rows, we will drop the data

In [ ]:
df = df.loc[~df.horsepower.isna(), :].copy()

# Finishing touch is the categorical features encoding
* We will need to deal with *origin* and *car_name* features
    * Why is *origin* categorical? Isn't it numerical value already? What's the catch?

### car_name is problematic beacause we have quite a few brands so one-hot encoding would add too many columns
* We will drop the feature

In [ ]:
df['car_name'].apply(lambda x: x.split(' ')[0]).value_counts()

In [ ]:
df = df.drop('car_name', axis=1)

In [ ]:
df['origin'] = df['origin'].replace({1: 'USA', 2: 'EUR', 3: 'JAP'})

In [ ]:
df = pd.get_dummies(df, columns=['origin'], prefix=['origin_'])

In [ ]:
df.head()

## Split the data into input and output part

In [ ]:
X, y = df.drop('mpg', axis=1), df.mpg

In [ ]:
X.shape, y.shape

## Do the train/test in ratio 80:20

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Now we can create a baseline model now
* We will use traditional linear regression

## Which model is better explainable from your point of view?
* ANN or Lin. regression? Why?

![Meme01](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_meme_reg_01.jpg?raw=true)

## We will use just *horsepower* and *model_year* features because the high correlation values

In [ ]:
alg = LinearRegression()
alg.fit(X_train.loc[:, ['horsepower', 'model_year']], y_train)
y_pred = alg.predict(X_test.loc[:, ['horsepower', 'model_year']])

## We can evaluate the model now

* We evaluated quality of classification models using its accuracy in the previous lectures.

* Regression tasks are not different - we have to evaluate model quality as well, but now we use different types of metrics
    * Most basic ones are MAE, RMSE
    * There are many more metrics - R2, MAPE, sMAPE etc
    
### 🔎 Can you define any of the metrics using mathematical formula?

* We have prepared the evaluation functions API for you so you can evaluate all the metrics in one function call

In [ ]:
df_pred = pd.DataFrame({'y_true': y_test, 'y_pred': y_pred})
compute_metrics(df_pred)

# Now we can create our first deep learning model and compare it to the baseline
* The ANN model can use more features as it is designed for bigger datasets and multicolinearity is not so big issue as in the LR case
* We will start with a raw data
* The evaluation step is the same

![Meme02](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_meme_reg_04.jpg?raw=true)

## 🔎 Why do we use *linear* activation in the output layer?

# 📒 NOTE for Task 2: This is the benchmark model

In [ ]:
inp = keras.layers.Input(shape=(X_train.shape[1]))
                         
hidden_1 = keras.layers.Dense(128, activation='relu')(inp)
hidden_2 = keras.layers.Dense(32, activation='relu')(hidden_1)

out = keras.layers.Dense(1, activation='linear')(hidden_2)

model = keras.Model(inp, out)

model.compile(loss=keras.losses.MeanSquaredError(),  
              optimizer=keras.optimizers.RMSprop(), 
              metrics=[keras.metrics.MeanAbsoluteError(), keras.metrics.MeanAbsolutePercentageError()])
model.summary()

## Train the model

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='weights.best.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

history = model.fit(X_train, y_train, validation_split=0.2, callbacks=[model_checkpoint_callback], batch_size=8, epochs=100)

## We can compute the same metrics as in the LR model case
* Is the model better?
    * Why?
* Why do we use .ravel() function?

In [ ]:
model.load_weights("weights.best.hdf5")

y_pred = model.predict(X_test).ravel()

df_pred = pd.DataFrame({'y_true': y_test, 'y_pred': y_pred})
compute_metrics(df_pred)

## It is very good practice to check the loss function values of train/validation data during the training and not only the metrics
* Do you see any issue with the val_loss?

In [ ]:
show_history_loss(history)

## The loss function plot show clear instability of learning
* This is a big issue in the regression tasks and it is pretty common one
* It is caused by the features magnitude differences
* We can solve the matter with feature scaling (normalization)
* A https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization layer can be used for solving the matter

### Why is magnitude difference an issue?

* You can see that the gradient of the slope is orders of magnitude larger than the intercept.

![Grad01](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_reg_noscale.png?raw=true)

* If we take a look at the one optimization step values change you can see that only the slope changed in value (we see a vertical line in the plot above, with no change in the intercept parameter). 
    * That’s because the slope gradient is way bigger than the intercept gradient.
    * Gradient actually points in the direction of steepest ascent.
    * Gradient is the vector of all partial derivatives of the loss function with respect to all the model weights.
        * **Basically these values will tell you in which direction (+ or - delta) and how much you should change the individual weights values to lower the loss function value**
        * The amount we adjust our slope each iteration is controlled by a *learning rate* parameter
    
![Grad02](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_reg_noscale_grad.png?raw=true)

### There are a few ways we can solve our problem above. The most common way is to simply scale your features before gradient descent.

![Grad03](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_reg_scale.png?raw=true)

* We can see that not the optimization process is not stuck and computed gradients in the individual steps points in the right direction.

![Grad04](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_reg_scale_grad.png?raw=true)


* **I recommend visiting https://www.tomasbeuzen.com/deep-learning-with-pytorch/chapters/chapter1_gradient-descent.html for more details about the topic**

# We will normalize the data now and try to fit the model again

In [ ]:
norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(X_train)

## We can take a look at the mean and variance used in the normalization process for each feature

In [ ]:
print('Mean: ', np.array(norm_layer.variables[0]))
print('Variance: ', np.array(norm_layer.variables[1]))

In [ ]:
inp = keras.layers.Input(shape=(X_train.shape[1]))
norm = norm_layer(inp)                  
hidden_1 = keras.layers.Dense(128, activation='relu')(norm)
hidden_2 = keras.layers.Dense(32, activation='relu')(hidden_1)

out = keras.layers.Dense(1, activation='linear')(hidden_2)

model = keras.Model(inp, out)

model.compile(loss=keras.losses.MeanSquaredError(),  
              optimizer=keras.optimizers.RMSprop(), 
              metrics=[keras.metrics.MeanAbsoluteError(), keras.metrics.MeanAbsolutePercentageError()])
model.summary()

## Train the model

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='weights.best.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

history = model.fit(X_train, y_train, validation_split=0.2, callbacks=[model_checkpoint_callback], batch_size=8, epochs=100)

In [ ]:
model.load_weights("weights.best.hdf5")

y_pred = model.predict(X_test).ravel()

df_pred = pd.DataFrame({'y_true': y_test, 'y_pred': y_pred})
compute_metrics(df_pred)

## Do you see any difference?

* Normalization can help training of our neural networks as the different features are on a similar scale, which helps to stabilize the gradient descent step, allowing us to use larger learning rates or help models converge faster for a given learning rate.

In [ ]:
show_history_loss(history)

## We can transform the output as well
* There are multiple scaling options
    * MinMax, Std. scale, Log, BoxCox, ...
    
### We will test *MinMaxScaler* into (-1;1) range

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
y_train_scaled = scaler.fit_transform(np.array(y_train).reshape((-1, 1))).ravel()
y_test_scaled = scaler.transform(np.array(y_test).reshape((-1, 1))).ravel()

In [ ]:
y_train_scaled[:10]

# ⚠ Beware the output layer range of the used activation function ⚠
### If there is a mismatch between range of the output activation function and output variable range, the model won't work as ANN may not be able to produce numbers from the correct range

### We will try to produce such an issue now
* We will set *sigmoid* as an output func. thus we won't be able to output negative numbers

# ☣ ANTI-EXAMPLE ☣

In [ ]:
inp = keras.layers.Input(shape=(X_train.shape[1]))
norm = norm_layer(inp)                  
hidden_1 = keras.layers.Dense(128, activation='relu')(norm)
hidden_2 = keras.layers.Dense(32, activation='relu')(hidden_1)

out = keras.layers.Dense(1, activation='sigmoid')(hidden_2)

model = keras.Model(inp, out)

model.compile(loss=keras.losses.MeanSquaredError(),  
              optimizer=keras.optimizers.RMSprop(), 
              metrics=[keras.metrics.MeanAbsoluteError(), keras.metrics.MeanAbsolutePercentageError()])
model.summary()

## Train the model

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='weights.best.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

history = model.fit(X_train, y_train_scaled, validation_split=0.2, callbacks=[model_checkpoint_callback], batch_size=8, epochs=100)

In [ ]:
model.load_weights("weights.best.hdf5")

y_pred = model.predict(X_test).ravel()

df_pred = pd.DataFrame({'y_true': y_test_scaled, 'y_pred': y_pred})
compute_metrics(df_pred)

## Now we can transfer the data back

In [ ]:
y_pred = scaler.inverse_transform(y_pred.reshape((-1, 1))).ravel()
y_pred[:10]

In [ ]:
df_pred = pd.DataFrame({'y_true': y_test, 'y_pred': y_pred})
compute_metrics(df_pred)

# Plot of the y_test vs. y_pred
* Do you see the issue?
* How should the ideal plot look like?

In [ ]:
sns.scatterplot(x=y_test, y=y_pred)

In [ ]:
show_history_loss(history)

# ✅ Now we will try to fix the issue and replace sigmoid function with the correct one ✅
* What function can we use? Why?

In [ ]:
inp = keras.layers.Input(shape=(X_train.shape[1]))
norm = norm_layer(inp)                  
hidden_1 = keras.layers.Dense(128, activation='relu')(norm)
hidden_2 = keras.layers.Dense(32, activation='relu')(hidden_1)

out = keras.layers.Dense(1, activation='tanh')(hidden_2)

model = keras.Model(inp, out)

model.compile(loss=keras.losses.MeanSquaredError(),  
              optimizer=keras.optimizers.RMSprop(), 
              metrics=[keras.metrics.MeanAbsoluteError(), keras.metrics.MeanAbsolutePercentageError()])
model.summary()

## Train the model

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='weights.best.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

history = model.fit(X_train, y_train_scaled, validation_split=0.2, callbacks=[model_checkpoint_callback], batch_size=8, epochs=100)

In [ ]:
model.load_weights("weights.best.hdf5")

y_pred = model.predict(X_test).ravel()

df_pred = pd.DataFrame({'y_true': y_test_scaled, 'y_pred': y_pred})
compute_metrics(df_pred)

## Now we can transfer the data back

In [ ]:
y_pred = scaler.inverse_transform(y_pred.reshape((-1, 1))).ravel()
y_pred[:10]

In [ ]:
df_pred = pd.DataFrame({'y_true': y_test, 'y_pred': y_pred})
compute_metrics(df_pred)

# Plot of the y_test vs. y_pred
* Is it better?

In [ ]:
sns.scatterplot(x=y_test, y=y_pred)

## The convergence was quite fast
* We can see that there is an issue with the val_loss stability as the changes are very low now

In [ ]:
show_history_loss(history)

## Tasks for the lecture (2p)

1) Try to use [PowerTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html) for the output values in a similar manner as the MinMaxScaler - **(1p)**
    - When do we use it? Why?
    - If you wanted to guess if it helps, what do you think? 
        * Plot histogram of the output (*mpg*), you can make an educated guess based on it 🙂
    
2) Try to design your own network and beat the **benchmark** network used in the lecture - **(1p)**